In [80]:
import pandas as pd
import re

file_name = "Zoom_sample_transcript.xlsx"
xl = pd.ExcelFile(file_name)
df = xl.parse("Sheet1", index_col=None, header=None)
df.columns = ['raw']
df.drop(range(0,len(df),3), inplace=True)
df.reset_index(inplace=True, drop=True)

df_timestamp = df.iloc[0::2, :]
df_timestamp.reset_index(inplace=True, drop=True)

df_text = df.iloc[1::2, :]
df_text.reset_index(inplace=True, drop=True)

df = pd.concat([df_timestamp,df_text],axis=1)
df.columns = ['timestamp','text']


names = []
for index,row in df.iterrows():
    p = re.compile("^(\w+\s\w+):", re.I)
    name = p.search(row["text"])
    text = p.sub("",row["text"])
    df.loc[index]['text'] = text
    if name:
        names.append(name.group()[:-1])
    else:
        names.append("unclear")
    p2 = re.compile("\d\d:\d\d:\d\d.\d\d(?=\d)")
    inq_time = p2.search(row["timestamp"])
    df.loc[index]['timestamp'] = "["+inq_time.group()+"]"
df["names"]=names
df = df[["timestamp","names","text"]]
save_file_name = "cleaned_"+file_name
df.to_excel(save_file_name, index=False, header=False)